In [ ]:
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from apertools import plotting
from matplotlib import cm

# sns.set_style(style="white")
plotting.set_style(size=16)

RNG = np.random.default_rng()

%matplotlib inline
%load_ext autoreload
%autoreload 2

# SAR Constellation Figures

# Private Companies

In [ ]:
# Get table of small sat constellations
df = pd.read_html("https://www.newspace.im/")[0]
print(df.shape)
df.head()

In [ ]:
df.drop(["Image", "Funding", "Form factor"], axis=1, inplace=True)
df.head()

In [ ]:
# Grab the sar ones
sar_rows = df["Field"].str.lower().str.contains("sar", na=False)
df_sar = df[sar_rows]

# Drop too-early stage/bankrupt/not planned yet
df_sar = df_sar[~df_sar["First launch"].str.contains("\?")]
df_sar = df_sar[~df_sar["First launch"].str.contains("Cancelled")]
df_sar = df_sar[~df_sar["Launched\xa0/Planned network"].str.contains("\?")]


# split out "launched" and "planned network"
launched_planned = (
    df_sar["Launched\xa0/Planned network"].str.split("/", expand=True).astype(int)
)
df_sar.drop("Launched\xa0/Planned network", axis=1, inplace=True)
df_sar["Launched"] = launched_planned.loc[:, 0]
df_sar["Planned"] = launched_planned.loc[:, 1]

# Reorder cols
df_sar = df_sar[
    [
        "Organization",
        "Launched",
        "Planned",
        "First launch",
        "Field",
        "Technical and comments",
    ]
]

# add a shorter name
df_sar["short_name"] = df_sar["Organization"]
df_sar.loc[:, "short_name"] = df_sar["short_name"].str.split(" \(").str[0]

df_sar = df_sar.reset_index(drop=True)
df_sar

In [ ]:
df_sar.to_csv("new_space_sar_constellations.csv", index=False, encoding="utf-8")

In [ ]:
df_sar2.loc[6].short_name = 'China Electronics\nTechnology Group'
Tianjin Satcom Geohe Technologies
df_sar2.loc[6]

In [ ]:
# df_sar2 = df_sar[df_sar['Launched'] > 0].copy()
df_sar2 = df_sar

fig, ax = plt.subplots(figsize=(6, 6))
# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/broken_axis.html
# fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# fig.subplots_adjust(hspace=0.05)  # adjust space between axes


xlabels = df_sar2["short_name"].values
ax.bar(x=xlabels, height=df_sar2["Launched"])
ax.bar(x=xlabels, height=df_sar2["Planned"], bottom=df_sar2["Launched"])


xlabels[6] = "China Electronics\nTechnology Group"
xlabels[12] = "Tianjin Satcom\nGeohe Technologies"
ax.set_xticklabels(xlabels, rotation=90, ha="center")
ax.set_yscale("log", base=2)
ax.set_ylim((0.8, None))

plt.tight_layout()

In [ ]:
# fig, ax = plt.subplots(figsize=(6, 6))
# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/broken_axis.html
# fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 5))


mosaic = [[0], [1], [1], [1]]
fig = plt.figure(constrained_layout=True, figsize=(8, 5))
axes = fig.subplot_mosaic(
    mosaic,
    sharex=True,
    gridspec_kw={
        "hspace": 0.01,
    },
)
ax1 = axes[0]
ax2 = axes[1]
# fig.subplots_adjust(hspace=0.05)  # adjust space between axes

df_sar_sorted = df_sar.sort_values(["Launched", "Planned"], ascending=False)

xlabels = df_sar_sorted["short_name"].values
# First plot same on both axes
# cmap = cm.get_cmap('tab10')
cmap = sns.color_palette("deep")

bars_launched = ax1.bar(x=xlabels, height=df_sar_sorted["Launched"], color=cmap[0])
bars_planned = ax1.bar(
    x=xlabels,
    height=df_sar_sorted["Planned"],
    bottom=df_sar_sorted["Launched"],
    color=cmap[-2],
)
ax2.bar(x=xlabels, height=df_sar_sorted["Launched"], color=cmap[0])
ax2.bar(
    x=xlabels,
    height=df_sar_sorted["Planned"],
    bottom=df_sar_sorted["Launched"],
    color=cmap[-2],
)

# zoom-in / limit the view to different portions of the data
ax1.set_ylim(92, 100)  # outliers only
ax2.set_ylim(0, 60)  # most of the data

xlabels[6] = "China Electronics\nTechnology Group"
xlabels[12] = "Tianjin Satcom\nGeohe Technologies"
ax2.set_xticklabels(xlabels, rotation=90, ha="center")

# hide the spines between ax and ax2
ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop=False)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()


# Now, let's turn towards the cut-out slanted lines.
# We create line objects in axes coordinates, in which (0,0), (0,1),
# (1,0), and (1,1) are the four corners of the axes.
# The slanted lines themselves are markers at those locations, such that the
# lines keep their angle and position, independent of the axes size or scale
# Finally, we need to disable clipping.

d = 0.5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(
    marker=[(-1, -d), (1, d)],
    markersize=12,
    linestyle="none",
    color="k",
    mec="k",
    mew=1,
    clip_on=False,
)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)


ax1.legend([bars_launched, bars_planned], ["Launched", "Planned"], loc="upper left")

plt.minorticks_off()
plt.tight_layout()

In [ ]:
fig.savefig(
    "../figures/chapter3-sar/sar-private-constellations.pdf", bbox_inches="tight"
)

## Current count of private satellites

In [ ]:
columns = ["short_name", "Launched", "Planned"]


df_sar0 = df_sar[df_sar.Launched == 0]
df_sar1 = df_sar[df_sar.Launched > 0]

# xlabels[6] = 'China Electronics\nTechnology Group'
# xlabels[12] = 'Tianjin Satcom\nGeohe Technologies'

df = (
    df_sar[columns]
    .set_index("short_name")
    .sort_values(["Launched", "Planned"], ascending=False)
)
# df.append(df.sum(numeric_only=True), ignore_index=True)
# df.append(df.sum(numeric_only=True), ignore_index=True)
df = df.append(pd.Series(df.sum(), name="Total"))
df.index.name = "Company"
df

# Government missions

In [ ]:
df_govt = pd.read_csv("../sar-missions.csv", parse_dates=["Launched", "Ended"])
df_govt.drop(
    ["Site", "Satellite Revisit Time (days)", "Swath Width (km)"], axis=1, inplace=True
)

# cut future missions
future_idxs = df_govt["Launched"].str.lower() == "future"
df_govt = df_govt[~future_idxs]

# Fix a weird hypen in RADARSAT, idk if it's an excel thing or what
bad_hypen = df_govt[df_govt["Mission"] == "RADARSAT"].iloc[0].Satellite[-2]
df_govt["Satellite"] = df_govt["Satellite"].str.replace(bad_hypen, "-")

# replace with future date
ongoing_idxs = df_govt["Ended"] == "present"

df_govt["Ended"][ongoing_idxs] = (pd.Timestamp.today() + pd.DateOffset(days=90)).date()
df_govt["Ended"] = pd.to_datetime(df_govt["Ended"])
df_govt["Launched"] = pd.to_datetime(df_govt["Launched"])

# df_govt['span'] = df_govt['Ended'] - df_govt['Launched']
df_govt["Span"] = (df_govt["Ended"] - df_govt["Launched"]).dt.days

# ignore seasat for now to just get post-90s missions
df_govt = df_govt[df_govt["Launched"] > pd.to_datetime("1990-01-01")]
df_govt.reset_index(drop=True, inplace=True)

# Use Nation/Agency as legend key
df_govt["Legend"] = df_govt["Agency"] + " (" + df_govt["Nation"] + ")"
# For Legend simplicity, just call the SIR-X NASA instead of NASA/DLR/ASI
df_govt["Legend"] = df_govt["Legend"].replace("NASA/DLR (USA)", "NASA (USA)")

df_govt.head()

In [ ]:
wavelengths = {"X": 3, "C": 6, "L": 24}
df_govt["Wavelength"] = 0
for b, val in wavelengths.items():
    idxs = df_govt["Band"].str.contains(b)
    df_govt.loc[idxs, "Wavelength"] = val

df_govt.head()
df_govt = df_govt.sort_values(
    by=["Wavelength", "Launched"], axis=0, ascending=[False, True]
)
# df_govt = df_govt.sort_values(by=["Wavelength"], axis=0)
df_govt

In [ ]:
first_c_idx = (df_govt["Band"] == "C").values.argmax()
first_x_idx = (df_govt["Band"] == "X").values.argmax()
first_c_idx, first_x_idx

In [ ]:
df_govt["Legend"].unique(), len(df_govt["Legend"].unique())

In [ ]:
# Make each Nation a separate color
cmap = "set3"
cmap = "paired"
# cmap = "tab20"

x = np.linspace(0.0, 1.0, 50)
label_colors = np.unique(cm.get_cmap(plt.get_cmap(cmap))(x), axis=0)

color_df = df_govt.groupby("Nation").first().reset_index()[["Nation"]]
# Remove USA since the 2 missions are two short for colors
color_df = color_df[color_df.Nation != "USA"]


for row in color_df.itertuples():
    # print(row)
    df_govt.loc[df_govt["Nation"] == row.Nation, "color"] = row.Index
df_govt["color"] = df_govt["color"].astype(int)
# df_govt.head()

In [ ]:
sns.set_style()
# import proplot # for style
import matplotlib.ticker as ticker

fig, ax = plt.subplots(figsize=(10, 10))

bar_colors = label_colors[df_govt.color.values]
# ys = np.arange(3
bar_list = ax.barh(
    y=df_govt["Satellite"],
    width=df_govt["Span"],
    left=df_govt["Launched"],
    height=0.9,
    align="center",
    edgecolor="k",
    linewidth=2,
    # color="grey",
    color=bar_colors,
)


tick_dates = pd.date_range("1990-1-1", end=df_govt["Ended"].max(), freq="5Y")
ax.set_xticks(tick_dates)
ax.xaxis.set_major_formatter(ticker.FixedFormatter(tick_dates.strftime("%Y")))
ax.tick_params(
    axis="x",
    # rotation=70,
    labelsize="x-large",
)

ax.yaxis.tick_right()

xmin = pd.to_datetime("1988-1-1")
xmax = pd.Timestamp.today().date()
# give some x-limit space on the left for annotations of x/c/l
ax.set_xlim((xmin, xmax))
ax.set_ylim((-1, len(df_govt)))

ax.grid(visible=True, which="major")

# Add divider for X/C/L band missions
ax.hlines(
    xmin=xmin,
    xmax=xmax,
    y=[first_c_idx - 0.5, first_x_idx - 0.5],
    lw=3,
    color="black",
    linestyle="dashed",
)
ax.vlines(
    ymin=-5,
    ymax=len(df_govt) + 5,
    x=tick_dates.min() - pd.DateOffset(days=180),
    lw=3,
    color="black",
    linestyle="dashed",
)


# Add annotations for NASA ones (too thin to see color)
nasa_labels = ["(NASA/DLR, USA/Germany)", "(NASA, USA)"]
nasa_rows = df_govt.reset_index()[df_govt.Legend.str.contains("USA").values][
    ["Ended", "Legend"]
]
for lab, (idx, row) in zip(nasa_labels, nasa_rows.iterrows()):
    ax.annotate(
        lab,
        xy=(row.Ended + pd.DateOffset(days=180), idx),
        va="center",
        font="TeX Gyre Heros",
        size=11,
    )

# dd stars to denote freely available data
df_free = df_govt.reset_index().loc[df_govt.reset_index()["Freely Available"] == "Yes"]
free_markers = ax.scatter(
    df_free["Launched"] - pd.DateOffset(days=180), df_free.index, marker="*", s=150
)

# Add text denoting the X/C/L divides
xtext = pd.to_datetime("1989-4-1")
ax.text(
    xtext,
    1.8,
    "L-Band",
    ha="center",
    va="center",
    rotation=90,
    size=15,
    bbox=dict(
        lw=2,
        fc="w",
        ec="k",
    ),
)
ax.text(
    xtext,
    12,
    "C-Band",
    ha="center",
    va="center",
    rotation=90,
    size=15,
    bbox=dict(
        lw=2,
        fc="w",
        ec="k",
    ),
)
ax.text(
    xtext,
    26,
    "X-Band",
    ha="center",
    va="center",
    rotation=90,
    size=15,
    bbox=dict(
        lw=2,
        fc="w",
        ec="k",
    ),
)


# Make a dict of labels to de-duplicate per nation
by_label = dict(zip(df_govt["Legend"], bar_list))
by_label.pop("NASA (USA)")
by_label["Data Freely Available"] = free_markers
ax.legend(
    by_label.values(),
    by_label.keys(),
    fontsize="large",
    bbox_to_anchor=(0.6, 1.0),
    ncol=2,
)

# ax.xaxis.tick_params(which='minor', grid_alpha=0)
plt.minorticks_off()

In [ ]:
fig.savefig("../figures/chapter3-sar/sar-missions.pdf", bbox_inches="tight")

## Current count of government satellites

In [ ]:
df_govt[df_govt["Ended"] >= pd.Timestamp.today()].reset_index()

# 3.4 Hawaii example interferogram

In [ ]:
from apertools import constants, plotting, sario

In [ ]:
data_dir = "/Users/scott/Documents/Learning/phd-thesis/scratch/hawaii-examples/"
os.chdir(data_dir)

In [ ]:
# baseline = -16.6, but good stratified cone. also, pre-puuooo collapse
# ifgname = "20180408_20180420"
# baseline = -28, mauna loa kinda misshapen.
# ifgname = "20180408_20180502"

ifgname = "20180420_20180502"  # This one seems nicest overall

dem = sario.load("wgs84/igrams/elevation_looked.dem")
ifg = sario.load(f"wgs84/igrams/{ifgname}.int")
unw = sario.load(f"wgs84/igrams/{ifgname}.unw")

mask = ifg == 0
dem[mask] = 0
ifg[mask] = np.nan
unw[mask] = np.nan

In [ ]:
import rioxarray

unw_rio = rioxarray.open_rasterio(f"wgs84/igrams/{ifgname}.unw").sel(band=2)
unw_rio

In [ ]:
plotting.plot_ifg(ifg, figsize=(12, 3))
fig, ax = plt.subplots(figsize=(6, 6))
axim = ax.imshow(dem, cmap="terrain")
fig.colorbar(axim)

In [ ]:
plt.imshow(unw, cmap="RdBu")

# 3.5 Tropo noise Hawaii zoom

In [ ]:
s1, s2 = slice(220, 500), slice(380, 650)
axes = plotting.plot_ifg(ifg[s1, s2], figsize=(12, 3))

patch = unw[s1, s2].copy()
patch -= np.nanmean(patch)
axim = axes[-1].imshow(patch, cmap="RdBu")
plt.colorbar(axim)


fig, axes = plt.subplots(1, 2, figsize=(12, 3))
ax = axes[0]
axim = ax.imshow(dem[s1, s2], cmap="terrain")
fig.colorbar(axim)

ax = axes[1]

ax.scatter(dem[s1, s2].ravel(), patch.ravel(), s=0.2)
ax.set_xlabel("Elevation [m]")
ax.set_ylabel("Unwrapped phase [cm]")

In [ ]:
from apertools import latlon

lons, lats = latlon.grid(**sario.load("wgs84/igrams/dem.rsc"), sparse=True)
lons = lons.ravel()
lats = lats.ravel()
lons.shape

In [ ]:
pplt.rc["grid"] = False

In [ ]:
rowmax, colmax = np.unravel_index(dem.argmax(), dem.shape)  # Mauna Kea, 4200ish
# rowmax, colmax = 742, 624 # mauna loa, 4100ish
buf = 50
rows, cols = slice(rowmax - buf, rowmax + buf), slice(colmax - buf, colmax + buf)
bbox = unw_rio[rows, cols].rio.bounds()

# fig, axes = plt.subplots(1, 3, figsize=(14, 4),
#                         # sharex=True, sharey=True,
#                         )
fig = pplt.figure(refwidth=1.5)
axes = fig.subplots(ncols=3, proj=("pcarree", "pcarree", None))

ax = axes[0]
patch = unw[rows, cols].copy() * constants.PHASE_TO_CM_S1
patch -= patch.mean()
# axim = ax.imshow(patch , cmap='RdBu', vmin=-2, vmax=2)
# axim = ax.imshow(np.angle(ifg[rows, cols]), cmap='dismph', interpolation='nearest')
_, axim = plotting.map_img(
    patch, bbox=bbox, pad_pct=0.0, ax=ax, cmap="RdBu", vmin=-2, vmax=2
)  # , crs=None, **imshow_kwargs):
cbar = fig.colorbar(axim, ax=ax)
cbar.set_label("[cm]")
ax.set_axis_off()
plotting.scale_bar(ax, ax.projection, 2, location=(0.8, 0.08))
ax.format(abc="(a)")


ax = axes[1]
axim = ax.imshow(dem[rows, cols], cmap="terrain")
_, axim = plotting.map_img(
    dem[rows, cols], bbox=bbox, pad_pct=0.0, ax=ax, cmap="terrain"
)
cbar = fig.colorbar(axim, ax=ax)
cbar.set_label("[m]")
ax.set_axis_off()
plotting.scale_bar(ax, ax.projection, 2, location=(0.8, 0.08))
ax.format(abc="(a)")


x, y = dem[rows, cols].ravel(), patch.ravel()

ax = axes[2]
# ax = fig.add_subplot(1, 3, 3)
ax.scatter(x / 1e3, y, s=0.2)
ax.set_xlabel("Elevation [km]")
ax.set_ylabel("Unwrapped phase [cm]")
ax.grid(True)
ax.minorticks_off()
ax.format(abc="(a)")

# fig.tight_layout()

In [ ]:
fig.savefig("../../figures/chapter3-sar/hawaii-strat-zoom.pdf")

In [ ]:
patch = unw[650:950, 900:1300].copy() * constants.PHASE_TO_CM_S1
patch -= np.nanmean(patch)
plt.imshow(patch)
plt.colorbar()

In [ ]:
x, y = dem[rows, cols].ravel(), unw[rows, cols].ravel() * constants.PHASE_TO_CM_S1

ax = axes[2]
ax.scatter(x, y, s=0.2)
ax.set_xlabel("Elevation [m]")
ax.set_ylabel("Unwrapped phase [cm]")

fig.tight_layout()
fig.save_fig("../hawaii-mauna-loa-stratified-noise")

In [ ]:
# Chronology: https://volcanoes.usgs.gov/vsc/file_mngr/file-179/Chronology%20of%20events%202018.pdf
# https://www.usgs.gov/volcanoes/kilauea/2018-lower-east-rift-zone-eruption-and-summit-collapse
# https://www.youtube.com/watch?v=sPVlcvDrxtQ
# https://www.usgs.gov/media/images/clear-conditions-puu-oo-provided-good-views
# https://www.usgs.gov/observatories/hvo/news/volcano-watch-7-months-no-lava-puu-oo-heralds-end-era

In [ ]:
from apertools import gps, gps_plots

fig, axes = gps_plots.plot_gps_enu(
    "PUOC",
    start_date="2018-04-01",
    end_date="2018-06-01",
    figsize=(10, 6),
    days_smooth=0,
)

# 3.5 WRF mitigation images from raider

In [ ]:
data_dir = "/Users/scott/Documents/Learning/phd-thesis/scratch/weather_images/"
os.chdir(data_dir)

In [ ]:
weather_files = sorted(glob.glob("wea*.tif"))
weather_files

In [ ]:
dem78 = rioxarray.open_rasterio("elevation_looked.dem").sel(band=1)
dem78

In [ ]:
import rasterio as rio

with rio.open(weather_files[0]) as src:
    bbox = tuple(src.bounds)
    extent = bbox[0], bbox[2], bbox[1], bbox[3]

In [ ]:
bbox, extent

In [ ]:
weather_arrs = []
for f in weather_files:
    a = sario.load(f)
    if len(a) == 3:
        weather_arrs.append(np.moveaxis(a, 0, 2))
    else:
        weather_arrs.append(a.squeeze())
# arr_dict = {
#     'calm': {
#         'ifg': unw_calm,
#         'raider': d_raider_unw_calm,
#         'rgb': rgb_calm,
#     },
#     'storm': {
#         'ifg': unw_storm,
#         'raider': d_raider_unw_storm,
#         'rgb': rgb_storm,
#     },
#     'wave': {
#         'ifg': unw_wave,
#         'raider': d_raider_unw_wave,
#         'rgb': rgb_wave,
#     },
# }
[f.shape for f in weather_arrs]

In [ ]:
import proplot as pplt

cmap = "RdBu"
# fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 9))
# axes = axes.ravel()
fig, axes = pplt.subplots(nrows=3, ncols=3, figsize=(9, 9), share=False)

for ax, arr, fname in zip(axes, weather_arrs, weather_files):
    type_, d1, d2 = fname.strip(".tif").split("_")[-3:]
    print(type_, d1, d2)
    if arr.ndim == 3:
        ax.imshow(arr)
        ax.set_axis_off()

    else:
        axim = ax.imshow(arr, cmap=cmap, extent=extent)
        ax.colorbar(axim, loc="r")
        # fig.colorbar(axim, ax=ax)
    ax.set_title(f"{type_, d1, d2}")

# ax.imshow(rgb_storm)

In [ ]:
weather_files

### Links to weather on the GOES days

(remember that it's like 1am UTC on 7-23, which is 7-22 near sunset)

1. Storm: https://www.wunderground.com/history/daily/us/tx/seminole/KGNC/date/2019-7-22
2. Heat wave: https://www.wunderground.com/history/daily/us/tx/kermit/KINK/date/2018-5-28
    - Other one SE of Kermit: https://www.wunderground.com/history/daily/us/tx/san-angelo/KSJT/date/2018-5-28

https://stackoverflow.com/questions/4563272/how-to-convert-a-utc-datetime-to-a-local-datetime-using-only-standard-library
```
In [15]: utctime
Out[15]: datetime.datetime(2014, 11, 4, 0, 51, 6, tzinfo=datetime.timezone.utc)

In [16]: utctime.astimezone(localtz)
Out[16]: datetime.datetime(2014, 11, 3, 18, 51, 6, tzinfo=backports.zoneinfo.ZoneInfo(key='America/Chicago'))
```

In [ ]:
from datetime import datetime

from backports import zoneinfo

localtz = zoneinfo.ZoneInfo("America/Chicago")
utctime = datetime.fromisoformat("2014-11-04 00:51:06+00:00")
acq_time_local = utctime.astimezone(localtz)
acq_time_local

In [ ]:
# inches mercury to hPa, https://en.wikipedia.org/wiki/Inch_of_mercury
inches_to_hpa = 33.7685

In [ ]:
!pwd

In [ ]:
# Fails for some reason, maybe needs selenium to scrape
# import requests
# pd.read_html("https://www.wunderground.com/history/daily/us/tx/kermit/KINK/date/2018-5-28")
# r = requests.get("https://www.wunderground.com/history/daily/us/tx/kermit/KINK/date/2018-5-28")

From https://www.engineeringtoolbox.com/relative-humidity-air-d_687.html

> φ = pw / pws 100% ,
> where φ = relative humidity [%],
> pw = vapor partial pressure [mbar],
> pws = saturation vapor partial pressure at the actual dry bulb temperature [mbar].

In [ ]:
df_rel_humid = pd.read_html(
    "https://www.engineeringtoolbox.com/relative-humidity-air-d_687.html"
)[0]
df_rel_humid.columns = ["C", "F", "pws"]
# col_tf = df_rel_humid.columns[1]
df_rel_humid.head()

In [ ]:
df_rel_humid.plot(x="F", y="pws")

In [ ]:
# def fahr_to_pws
def get_pws(fahr):
    xp = df_rel_humid["F"]
    fp = df_rel_humid["pws"]
    return np.interp(fahr, xp, fp)


get_pws(np.arange(7))

In [ ]:
pd.Timedelta(1, "h")

In [ ]:
pd.to_datetime?

In [ ]:
def fahr_to_kelvin(fahrenheit):
    return (fahrenheit + 459.67) / 1.8


def get_weather_df(fname):
    df = pd.read_html(fname)[-1]
    df.dropna(inplace=True, how="all")
    print(df.columns)
    df = df.loc[:, ["Time", "Temperature", "Pressure", "Humidity"]]

    df["Time"] = pd.to_datetime(df["Time"]).dt.tz_localize('America/Chicago')
    # df["Time"] = pd.to_datetime(df["Time"]).dt.time
    df["Temperature"] = df["Temperature"].str.strip(" °F").astype(float)
    df["Pressure"] = df["Pressure"].str.strip(" °in").astype(float)
    df["P"] = df["Pressure"] * inches_to_hpa

    df["Humidity"] = df["Humidity"].str.strip(" °%").astype(float)
    # 1 mbar == 1 hPa
    # partial waterr vapor pressure
    df["e"] = get_pws(df["Temperature"]) * (df["Humidity"] / 100)

    df["T"] = fahr_to_kelvin(df["Temperature"])

    k1 = 77.6
    df["N_hydr"] = k1 * df["P"] / df["T"]
    # df.tail(8)

    k2 = 23.3
    k3 = 3.75e5
    df["N_wet"] = k2 * (df["e"] / df["T"]) + k3 * (df["e"] / df["T"] ** 2)
    return df


df_hourly_wave = get_weather_df(
    "../Kermit, TX Weather History | Weather Underground.html"
)
df_hourly_wave_se = get_weather_df(
    "../San Angelo, TX Weather History | Weather Underground.html"
)
df_hourly_wave.head()

In [ ]:
# Change in pressure
(df_hourly_wave.P.max() - df_hourly_wave.P.min())

In [ ]:
# Change in temperature
(df_hourly_wave["T"].max() - df_hourly_wave["T"].min())

In [ ]:
# Change in partial water vapor
(df_hourly_wave["e"].max() - df_hourly_wave["e"].min()), (
    df_hourly_wave_se["e"].max() - df_hourly_wave_se["e"].min()
)

In [ ]:
# pd.to_datetime(df_hourly_wave["Time"])
# tt = df_hourly_wave["Time"].apply(lambda d: d.hour)
# tt

In [ ]:
acq_time_local

In [ ]:
import matplotlib.dates as mdates

acq_time_local = acq_time_local.replace(
    year=df_hourly_wave["Time"][0].year,
    month=df_hourly_wave["Time"][0].month,
    day=df_hourly_wave["Time"][0].day,
)

fig, axes = pplt.subplots(ncols=2, sharey=False)

ax = axes[0]
# df_hourly_wave.set_index("Time")[["N_hydr"]]
ax.plot(df_hourly_wave["Time"], df_hourly_wave["N_hydr"], label="NW")
ax.plot(df_hourly_wave_se["Time"], df_hourly_wave_se["N_hydr"], label="SE")

ylim = ax.get_ylim()
ax.vlines(acq_time_local, 0, 500, linestyle="--", color="black")
ax.set_ylim(ylim)
ax.format(ylabel="N_hydr", xlabel="hour", xformatter=mdates.DateFormatter("%H:%M:%S"))
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
ax.legend()


ax = axes[1]
# df_hourly_wave.set_index("Time")[["N_hydr"]]
ax.plot(df_hourly_wave["Time"], df_hourly_wave["N_wet"], label="NW")
ax.plot(df_hourly_wave_se["Time"], df_hourly_wave_se["N_wet"], label="SE")

ylim = ax.get_ylim()
ax.vlines(acq_time_local, 0, 500, linestyle="--", color="black")
ax.set_ylim(ylim)
ax.format(ylabel="N_hydr", xlabel="hour", xformatter=mdates.DateFormatter("%H:%M:%S"))
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
ax.legend()

fig.format(suptitle='Refractivity for 2018-5-28 during West Texas "wave"')

In [ ]:

fig, axes = pplt.subplots(ncols=2, sharey=False)

ax = axes[0]
ax.plot(df_hourly_wave_se["Time"], df_hourly_wave_se["N_hydr"] - df_hourly_wave["N_hydr"], label="SE - NW")

ylim = ax.get_ylim()
ax.vlines(acq_time_local, 0, 500, linestyle="--", color="black")
ax.set_ylim(ylim)
ax.format(ylabel="N_hydr", xlabel="hour", xformatter=mdates.DateFormatter("%H:%M:%S"))


ax = axes[1]
ax.plot(df_hourly_wave_se["Time"], df_hourly_wave_se["N_wet"] - df_hourly_wave["N_wet"], label="SE - NW")

ylim = ax.get_ylim()
ax.vlines(acq_time_local, 0, 500, linestyle="--", color="black")
ax.set_ylim(ylim)
ax.format(ylabel="N_hydr", xlabel="hour", xformatter=mdates.DateFormatter("%H:%M:%S"))


fig.format(suptitle='Refractivity difference from SE to NW corner for 2018-5-28 during West Texas "wave"')

**Summary**: a mass of dry air moved from the northwest (mountain area), dropping the $N$ refractivity. This sped up the radar propagation speed, which looks like "uplift" inn the upper right of the interferogram.

In [ ]:
# If this N were the same along the whole path...
Ndiff = 10
h = 693_000  # m
r0 = h / np.cos(np.deg2rad(35))
r0 * Ndiff * 1e-6

but that's too much. the N gets closer to 0 in space, this is just the ground difference

In [ ]:
def plot_ifg_corrections(idxs=slice(0, 3), cmap="RdBu"):
    # fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 9))
    # axes = axes.ravel()
    # fig, axes = pplt.subplots(nrows=1, ncols=3, figsize=(9, 9), share=False)

    fig = pplt.figure(share=False, refwidth=2.3)

    ifg, raider, rgb = weather_arrs[idxs]
    type_, d2, d1 = weather_files[idxs][0].strip(".tif").split("_")[-3:]
    print(type_, d1, d2)

    # vm = max(np.nanmax(np.abs(ifg)), np.nanmax(np.abs(raider)))
    vm = max(
        np.nanpercentile(np.abs(ifg), 99.9), np.nanpercentile(np.abs(raider), 99.9)
    )

    ax = fig.subplot(221, title=f"GOES: {d2}", abc="(a)")
    ax.imshow(rgb, extent=extent)
    # ax.set_axis_off()
    ax.minorticks_off()

    ax = fig.subplot(222, title=f"Ifg: {d1} - {d2}", abc="(a)")
    axim = ax.imshow(ifg, cmap=cmap, extent=extent, vmax=vm, vmin=-vm)
    ax.colorbar(axim, loc="r", label="[cm]")
    # fig.colorbar(axim, ax=ax)
    # ax.set_title(f"{type_, d1, d2}")

    ax = fig.subplot(223, title="GMAO predicted", abc="(a)")
    axim = ax.imshow(raider, cmap=cmap, extent=extent, vmax=vm, vmin=-vm)
    ax.colorbar(axim, loc="r", label="[cm]")
    # fig.colorbar(axim, ax=ax)
    # ax.set_title(f"{type_, d1, d2}")

    ax = fig.subplot(224, title="Corrected Ifg", abc="(a)")
    axim = ax.imshow(ifg - raider, cmap=cmap, extent=extent, vmax=vm, vmin=-vm)
    ax.colorbar(axim, loc="r", label="[cm]")
    # fig.colorbar(axim, ax=ax)
    # ax.set_title(f"{type_, d1, d2}")
    return fig


pplt.rc["tick.minor"] = False

In [ ]:
fig = plot_ifg_corrections(slice(0, 3))

In [ ]:
fig = plot_ifg_corrections(slice(3, 6))
fig.savefig("../../figures/chapter3-sar/figure_tropo_correct_storm.pdf", dpi=100)

In [ ]:
fig = plot_ifg_corrections(slice(6, 9))
fig.savefig("../../figures/chapter3-sar/figure_tropo_correct_wave.pdf", dpi=100)

In [ ]:
def rms(arr):
    return np.sqrt(np.mean(arr ** 2))

In [ ]:
aa, bb = np.nan_to_num(weather_arrs[0:2])
print(f"Calm noise: {aa.min():.2f}, {aa.max():.2f}, {aa.ptp():.2f}, {rms(aa)=:.2f}")
cc = aa - bb
print(f"ifg - gmao: {cc.min():.2f}, {cc.max():.2f}, {cc.ptp():.2f}, {rms(cc)=:.2f}")
print()

aa, bb = np.nan_to_num(weather_arrs[3:5])
print(f"Storm noise: {aa.min():.2f}, {aa.max():.2f}, {aa.ptp():.2f}, {rms(aa)=:.2f}")
cc = aa - bb
print(f"ifg - gmao: {cc.min():.2f}, {cc.max():.2f}, {cc.ptp():.2f}, {rms(cc)=:.2f}")
print()

aa, bb = np.nan_to_num(weather_arrs[6:8])
print(f"Wave noise: {aa.min():.2f}, {aa.max():.2f}, {aa.ptp():.2f}, {rms(aa)=:.2f}")
cc = aa - bb
print(f"ifg - gmao: {cc.min():.2f}, {cc.max():.2f}, {cc.ptp():.2f}, {rms(cc)=:.2f}")
print()

In [ ]:
pplt.rc["grid.inlinelabels"] = False

In [ ]:
# plt.imshow(aa)
bbox78 = dem78.rio.bounds()

import cartopy.crs as ccrs
from cartopy.mpl.ticker import (
    LatitudeFormatter,
    LatitudeLocator,
    LongitudeFormatter,
    LongitudeLocator,
)

In [ ]:
def plot_ifg_corrections_proj(
    idxs=slice(0, 3),
    cmap="RdBu",
    dms=False,
    refwidth=2,
    latticks=[31.0, 31.5, 32.0, 32.5],
    lonticks=[-104.0, -103.0, -102.0],
):
    # fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 9))
    # axes = axes.ravel()
    # fig, axes = pplt.subplots(nrows=1, ncols=3, figsize=(9, 9), share=False)

    # fig = pplt.figure(share=False, refwidth=2.3)
    fig = pplt.figure(refwidth=refwidth)
    # axes = fig.subplots(proj=(None, 'pcarree', 'pcarree', 'pcarree'))

    ifg, raider, rgb = weather_arrs[idxs]
    type_, d2, d1 = weather_files[idxs][0].strip(".tif").split("_")[-3:]
    print(type_, d1, d2)

    # vm = max(np.nanmax(np.abs(ifg)), np.nanmax(np.abs(raider)))
    vm = max(
        np.nanpercentile(np.abs(ifg), 99.9), np.nanpercentile(np.abs(raider), 99.9)
    )

    ax = fig.subplot(
        221, title=f"GOES: {d2}", abc="(a)", projection="pcarree", labels=True, dms=dms
    )
    ax.format(lonlocator=lonticks, latlocator=latticks)
    # ax.imshow(rgb, extent=extent)
    _, axim = plotting.map_img(rgb, bbox=bbox78, pad_pct=0.0, ax=ax)
    # ax.set_xticks([-104, -103, -102], crs=ccrs.PlateCarree())
    # ax.set_yticks([30, 31], crs=ccrs.PlateCarree())

    ax = fig.subplot(
        222,
        title=f"Ifg: {d1} - {d2}",
        abc="(a)",
        projection="pcarree",
        labels=True,
        dms=dms,
    )
    ax.format(lonlocator=lonticks, latlocator=latticks)
    # axim = ax.imshow(ifg, cmap=cmap, extent=extent, vmax=vm, vmin=-vm)
    _, axim = plotting.map_img(
        ifg, bbox=bbox78, pad_pct=0.0, ax=ax, cmap=cmap, vmax=vm, vmin=-vm
    )
    # ax.set_xticks([-104, -103, -102], crs=ccrs.PlateCarree())
    # ax.set_yticks([31, 32], crs=ccrs.PlateCarree())
    cbar = ax.colorbar(axim, loc="r", label="[cm]")
    # fig.colorbar(axim, ax=ax)
    plotting.scale_bar(ax, ax.projection, 50, location=(0.8, 0.08))

    ax = fig.subplot(
        223,
        title="GMAO predicted",
        abc="(a)",
        projection="pcarree",
        labels=True,
        dms=dms,
    )
    ax.format(lonlocator=lonticks, latlocator=latticks)
    # axim = ax.imshow(raider, cmap=cmap, extent=extent, vmax=vm, vmin=-vm)
    _, axim = plotting.map_img(
        raider, bbox=bbox78, pad_pct=0.0, ax=ax, cmap=cmap, vmax=vm, vmin=-vm
    )
    # ax.set_xticks([-104, -103, -102], crs=ccrs.PlateCarree())
    # ax.set_yticks([30, 31], crs=ccrs.PlateCarree())
    ax.colorbar(axim, loc="r", label="[cm]")
    plotting.scale_bar(ax, ax.projection, 50, location=(0.8, 0.08))

    ax = fig.subplot(
        224,
        title="Corrected Ifg",
        abc="(a)",
        projection="pcarree",
        labels=True,
        dms=dms,
    )
    # axim = ax.imshow(ifg - raider, cmap=cmap, extent=extent, vmax=vm, vmin=-vm)
    ax.format(lonlocator=lonticks, latlocator=latticks)
    _, axim = plotting.map_img(
        ifg - raider, bbox=bbox78, pad_pct=0.0, ax=ax, cmap=cmap, vmax=vm, vmin=-vm
    )
    ax.colorbar(axim, loc="r", label="[cm]")
    plotting.scale_bar(ax, ax.projection, 50, location=(0.8, 0.08))
    # ax.set_xticks([-104, -103, -102], crs=ccrs.PlateCarree())
    # ax.set_yticks([30, 31], crs=ccrs.PlateCarree())
    return fig


pplt.rc["tick.minor"] = False

In [ ]:
fig = plot_ifg_corrections_proj(slice(3, 6))
fig.savefig("../../figures/chapter3-sar/figure_tropo_correct_storm.pdf", dpi=100)

In [ ]:
fig = plot_ifg_corrections_proj(slice(6, 9))
fig.savefig("../../figures/chapter3-sar/figure_tropo_correct_wave.pdf", dpi=100)

# 3.7 Section: LOS decomp 

In [ ]:
los_enu78 = rioxarray.open_rasterio("../los_enu/los_enu_path78.tif")
los_enu85 = rioxarray.open_rasterio("../los_enu/los_enu_path85.tif")

los_enu78

In [ ]:
mask78 = np.isnan(sario.load("../los_enu/path78.tif", band=1))
# mask85 = sario.load("../los_enu/path85.tif", band=1)
mask85 = sario.load("../los_enu/path85.int") == 0

In [ ]:
# plt.imshow(mask78, cmap='viridis')
# plt.imshow(mask85, cmap='viridis')
los_enu78.values[:, mask78] = np.nan
los_enu85.values[:, mask85] = np.nan

In [ ]:
los_enu85[0][: mask78.shape[0]].rio.bounds()

In [ ]:
los_enu78[0][: mask78.shape[0]].rio.bounds()

In [ ]:
cmap = "viridis"
dms = False
refwidth = (2,)
latticks = [31.0, 31.5, 32.0, 32.5]
lonticks = [-104.0, -103.0, -102.0]
titles = ["East", "North", "Up"]
scale_loc = (0.7, 0.08)
bbox = los_enu78[0].rio.bounds()

# fig = pplt.figure(share=False, refwidth=2.3)
fig = pplt.figure(refwidth=1.8)
axes = fig.subplots(nrows=2, ncols=3, proj="pcarree")  # , refwidth=refwidth)
axes.format(
    abc="(a)",
    labels=True,
    dms=dms,
    lonlocator=lonticks,
    latlocator=latticks,
    leftlabels=("Ascending (Path 78)", "Descending (Path 85)"),
)

for b, ax, t in zip(los_enu78, axes[:3], titles):
    # ax = fig.subplot(223, title='GMAO predicted', abc="(a)", projection='pcarree', labels=True, dms=dms)
    # ax.format(lonlocator=lonticks, latlocator=latticks)
    # axim = ax.imshow(raider, cmap=cmap, extent=extent, vmax=vm, vmin=-vm)
    _, axim = plotting.map_img(
        b, bbox=bbox, pad_pct=0.0, ax=ax, cmap=cmap
    )  # , vmax=vm, vmin=-vm)
    ax.colorbar(axim, loc="r")
    ax.set_title(t)
    if t == "East":
        plotting.scale_bar(ax, ax.projection, 50, location=scale_loc)


for b, ax, t in zip(los_enu85, axes[3:], titles):
    _, axim = plotting.map_img(
        b[: mask78.shape[0]], bbox=bbox, pad_pct=0.0, ax=ax, cmap=cmap
    )  # , vmax=vm, vmin=-vm)
    ax.colorbar(axim, loc="r")
    ax.set_title(t)
    if t == "East":
        plotting.scale_bar(ax, ax.projection, 50, location=scale_loc)

In [ ]:
fig.savefig("../../figures/chapter3-sar/figure_los_enu_coeffs.pdf", dpi=120)

## injection ascending/descending

In [ ]:
!pwd

In [ ]:
asc = rioxarray.open_rasterio("../injection_ascending.tif").sel(band=1)
desc = rioxarray.open_rasterio("../injection_descending.tif").sel(band=1)
asc

In [ ]:
cmap = "seismic_wide_y_r"
dms = False
refwidth = (2,)
latticks = [31.75, 31.80]
lonticks = [-103.30, -103.28]
titles = ["East", "North", "Up"]
scale_loc = (0.7, 0.08)
bbox = asc.rio.bounds()

# fig = pplt.figure(share=False, refwidth=2.3)
fig = pplt.figure(refwidth=1.5)
axes = fig.subplots(nrows=1, ncols=2, proj="pcarree")  # , refwidth=refwidth)
axes.format(
    labels=True,
    dms=dms,
    # abc="(a)",
    # lonlocator=lonticks,
    latlocator=latticks,
    # leftlabels=('Ascending (Path 78)', 'Descending (Path 85)'),
)

ax = axes[0]
_, axim = plotting.map_img(
    asc, bbox=bbox, pad_pct=0.0, ax=ax, cmap=cmap, vmax=0.5, vmin=-5.5
)
ax.colorbar(axim, loc="r", label="[cm]")
ax.set_title("Ascending (Path 78)")
ax = axes[1]
_, axim = plotting.map_img(
    desc, bbox=bbox, pad_pct=0.0, ax=ax, cmap=cmap, vmax=0.5, vmin=-5.5
)
ax.colorbar(axim, loc="r", label="[cm]")
ax.set_title("Descending (Path 85)")
# ax.set_title(t)
# plotting.scale_bar(ax, ax.projection, 50, location=scale_loc)


# for b, ax, t in zip(los_enu85, axes[3:], titles):
#     _, axim = plotting.map_img(b[:mask78.shape[0]], bbox=bbox, pad_pct=0.0, ax=ax, cmap=cmap) #, vmax=vm, vmin=-vm)
#     ax.colorbar(axim, loc='r')
#     ax.set_title(t)
#     if t == "East":
#         plotting.scale_bar(ax, ax.projection, 50, location=scale_loc)

In [ ]:
fig.savefig("../../figures/chapter3-sar/ch3-injection-asc-desc.pdf", dpi=120)